# Step 1: fine-tune LLM using top result from (fixed) ranker

In [2]:
# NB: amulet ate some log output (?)
def launch():
    import os
    import StepOne
    import torch

    os.environ['MODEL_TYPE'] = 'base'
    os.environ['BATCH_SIZE'] = '16'
    
    world_size = torch.cuda.device_count()
    torch.multiprocessing.spawn(StepOne.step_one,
                                args=(world_size,),
                                nprocs=world_size,
                                join=True)
    
launch()

******** augment = 4 max_iteration = 20 model_type = base *********
n                 iter (since)         4 loss (since)       4 rouge1 (since)   4 rouge1 ema (since) 4 rouge1 (dev) (since)      dt
1                0.000 (0.000)          3.523 (3.523)          0.177 (0.177)          0.177 (0.177)          0.000 (0.000)  20.5 s
2                0.000 (0.000)          3.432 (3.341)          0.161 (0.144)          0.148 (0.120)          0.000 (0.000)  34.8 s
16               0.000 (0.000)          3.185 (3.180)          0.162 (0.149)          0.163 (0.155)          0.000 (0.000)  2.86 m
32               0.000 (0.000)          3.105 (3.025)          0.171 (0.180)          0.170 (0.176)          0.000 (0.000)  5.47 m
64               0.000 (0.000)          3.059 (3.009)          0.172 (0.174)          0.170 (0.170)          0.166 (0.166)  10.7 m
128              0.000 (0.000)          3.059 (0.000)          0.172 (0.000)          0.170 (0.000)          0.171 (0.171)  17.6 m
183            

# Step 2: learn ranker using (fixed pre-finetuned) task LLM

In [ ]:
def launch():
    import os
    import StepTwo
    import torch

    os.environ['MODEL_TYPE'] = os.environ.get('MODEL_TYPE', 'base')
    os.environ['BATCH_SIZE'] = os.environ.get('BATCH_SIZE', '2')
    os.environ['LEARN_BATCH_SIZE'] = os.environ.get('LEARN_BATCH_SIZE', '2')
    os.environ['GRAD_FREE_BATCH_SIZE'] = os.environ.get('GRAD_FREE_BATCH_SIZE', '16')
    os.environ['STEP1_ITER'] = os.environ.get('STEP1_ITER', '17_augment4')
    
    world_size = torch.cuda.device_count()
    torch.multiprocessing.spawn(StepTwo.step_two,
                                args=(world_size,),
                                nprocs=world_size,
                                join=True)
    
launch()

1               0.000 (0.000)         0.398 (0.398)         0.273 (0.273)         0.000 (0.000)        21.500 (21.500)  1.06 m
2               0.000 (0.000)         0.426 (0.454)         0.207 (0.142)         0.000 (0.000)        24.750 (28.000)  1.39 m
4               0.000 (0.000)         0.485 (0.545)         0.229 (0.250)         0.000 (0.000)        23.375 (22.000)  1.89 m
8               0.000 (0.000)         0.466 (0.446)         0.208 (0.188)         0.000 (0.000)        18.250 (13.125)  3.17 m
16              0.000 (0.000)         0.460 (0.454)         0.224 (0.239)         0.000 (0.000)        20.250 (22.250)  5.76 m
32              0.000 (0.000)         0.406 (0.352)         0.188 (0.153)         0.000 (0.000)        18.766 (17.281)  9.78 m
64              0.000 (0.000)         0.462 (0.518)         0.202 (0.216)         0.000 (0.000)        18.133 (17.500)  17.6 m
128             0.000 (0.000)         0.456 (0.450)         0.206 (0.210)         0.000 (0.000)        14.434 (